In [ ]:

from numpy.random import seed
seed(13)
from tensorflow import set_random_seed
set_random_seed(13)

from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, MaxPooling3D, Dropout, Embedding, SimpleRNN, LSTM, GRU
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.models import model_from_json 
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from tempfile import TemporaryFile
from tqdm import tqdm
from operator import itemgetter
import numpy as np
import os
import cv2             
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
import shutil
import os
import pickle

# Load Data

In [ ]:
# %load conll_dictorizer.py
"""
CoNLL 2009 file readers and writers for the parts of speech.
Version with a class modeled as a vectorizer
"""
__author__ = "Pierre Nugues"

import regex as re


def save(file, corpus_dict, column_names):
    """
    Saves the corpus in a file
    :param file:
    :param corpus_dict:
    :param column_names:
    :return:
    """
    with open(file, 'w') as f_out:
        for sentence in corpus_dict:
            sentence_lst = []
            for row in sentence:
                items = map(lambda x: row.get(x, '_'), column_names)
                sentence_lst += '\t'.join(items) + '\n'
            sentence_lst += '\n'
            f_out.write(''.join(sentence_lst))


class Token(dict):
    pass


class CoNLLDictorizer:

    def __init__(self, column_names, sent_sep='\n\n', col_sep=' +'):
        self.column_names = column_names
        self.sent_sep = sent_sep
        self.col_sep = col_sep

    def fit(self):
        pass

    def transform(self, corpus):
        corpus = corpus.strip()
        sentences = re.split(self.sent_sep, corpus)
        return list(map(self._split_in_words, sentences))

    def fit_transform(self, corpus):
        return self.transform(corpus)

    def _split_in_words(self, sentence):
        rows = re.split('\n', sentence)
        return [Token(dict(zip(self.column_names,
                               re.split(self.col_sep, row))))
                for row in rows]


if __name__ == '__main__':
    
    BASE = os.getcwd()
    train_file = os.path.join(BASE, 'datasets/train.txt')

    column_names = ['id', 'form', 'lemma', 'cpos', 'pos', 'feats']
    train = open(train_file).read().strip()
    conll_dict = CoNLLDictorizer(column_names, col_sep='\t')
    train_dict = conll_dict.transform(train)

    print(train_dict[0])
    print(train_dict[0][0])
    print(type(train_dict[0][0]))
    #print(train_dict[0][0]['form'])
    print(train_dict[1])
    tok = Token({'id': '1', 'form': 'La', 'lemma': 'el', 'cpos': 'd', 'pos': 'da', 'feats': 'num=s|gen=f'})
    print(tok['form'])
    print('form' in tok)

    save('out', train_dict, column_names)

    tok_dict = {'id': '1', 'form': 'La', 'lemma': 'el', 'cpos': 'd', 'pos': 'da', 'feats': 'num=s|gen=f'}
    tok_dict2 = {'id': '1', 'form': 'La', 'lemma': 'el', 'cpos': 'd', 'pos': 'da', 'feats': 'num=s|gen=f'}

    tok_set = set(tok_dict)
    print(tok_set)

    tok_set = tok_set.union(tok_dict2)
    #print(tok_set)

    #print(tok.keys())

    # exit()
    word_set = set()
    word_set = set(tok_dict.values())
    #print(list(word_set))

    word_set = set()
    word_set = set(tok.values())
    #print(list(word_set))

    word_set = set()
    word_set.update(tok.values())
    #print(list(word_set))

    word_set = set()
    #print("Token value:", tok.values())
    word_set = word_set.union(set(tok.values()))
    #print(list(word_set))

In [ ]:
# %load datasets.py
from conll_dictorizer import CoNLLDictorizer, Token
import os

def load_conll2009_pos():
    train_file = 'datasets\train.txt'
    dev_file = 'datasets\valid.txt'
    test_file = 'datasets\test.txt'
    test2_file = 'simple_pos_test.txt'

    column_names = ['id', 'form', 'lemma', 'plemma', 'pos', 'ppos']

    train_sentences = open(train_file).read().strip()
    dev_sentences = open(dev_file).read().strip()
    test_sentences = open(test_file).read().strip()
    test2_sentences = open(test2_file).read().strip()
    return train_sentences, dev_sentences, test_sentences, column_names

def load_conll2003_en():
    BASE_DIR = os.getcwd()
    train_file = BASE_DIR + '/datasets/train.txt'
    dev_file = BASE_DIR + '/datasets/valid.txt'
    test_file = BASE_DIR + '/datasets/test.txt'
    column_names = ['form', 'ppos', 'pchunk', 'ner']
    train_sentences = open(train_file).read().strip()
    dev_sentences = open(dev_file).read().strip()
    test_sentences = open(test_file).read().strip()
    return train_sentences, dev_sentences, test_sentences, column_names


if __name__ == '__main__':
    train_sentences, dev_sentences, test_sentences, column_names = load_conll2003_en()

    conll_dict = CoNLLDictorizer(column_names, col_sep=' +')
    train_dict = conll_dict.transform(train_sentences)
    val_dict = conll_dict.transform(dev_sentences)
    test_dict = conll_dict.transform(test_sentences)
    print(train_dict[0])
    print(train_dict[1])

In [ ]:
def load_glove(file):
    embeddings_dict = {}
    glove = open(file, encoding='utf-8')
    
    for line in glove:
        line = line.strip().split()
        word = line[0]
        embedding_vec_word = np.array(line[1:], dtype='float32')
        embeddings_dict[word] = embedding_vec_word
        
    glove.close()
    return embeddings_dict

# Load and Save files

In [ ]:
def load_file(file_name):
    with open('files/' + file_name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
    return obj

In [ ]:
def save_file(file_name, file):
    with open('files/' + file_name + '.pkl', 'wb') as f:
        pickle.dump(file, f)

In [ ]:
embeddings_dict = load_file('embeddings_dict')

# Data preprocessing

In [ ]:
# Train_dict is a list of lists of dictionaries
def extract_features(train_dict):
    X, y = [], []
    
    for sentence in train_dict:
        X_sentence = []
        y_sentence = []
        for word in sentence:
            w = word['form'].lower()
            n = word['ner']
            X_sentence.append(w)
            y_sentence.append(n)
    
        X.append(X_sentence)
        y.append(y_sentence)
    
    return X, y

#### Extract words and ner tags - X, Y

In [ ]:
X, y = extract_features(train_dict)
print('Sentence words: ', X[1])
print('Sentence NER: ', y[1])

#### Create vocabularies

In [ ]:
def create_vocabulary(X, WORDS=True):
    X_vocabulary = set()
    
    if WORDS:
        X_vocabulary.add("UNKNOWN_WORD")
        
    for sentence in X:
        for word in sentence:
            X_vocabulary.add(word)
    
    return sorted(list(X_vocabulary))

In [ ]:
X_vocabulary = create_vocabulary(X, WORDS=True)
print("Vocabulary size WORDS: ", len(X_vocabulary))

In [ ]:
y_vocabulary = create_vocabulary(y, WORDS=False)
print("Vocabulary size NER: ", len(y_vocabulary))
nbr_of_classes = len(y_vocabulary) + 2

In [ ]:
y_vocabulary

#### Add words from GloVe

In [ ]:
for word in embeddings_dict.keys():
    X_vocabulary.append(word)

X_vocabulary = sorted(list(set(X_vocabulary)))
total_word_count = len(X_vocabulary)
print('Words in the vocabulary total, X_vocabulary:', total_word_count)

#### Create indices and inverted indices

In [ ]:
def create_indices(X):
    return dict(enumerate(X), start=2) # 0 is padding, 1 is unknown

In [ ]:
i = 2
X_indices_to_words = {}

for w in X_vocabulary:
    X_indices_to_words[i] = w
    i += 1

y_indices_to_len = {}
y_indices_to_len[0] = 'O' #PADDING
y_indices_to_len[1] = 'UNKNOWN_WORD'
i = 2

for l in y_vocabulary:
    if l != 'O':
        y_indices_to_len[i] = l
        i += 1
        
#X_indices_to_len = dict(enumerate(X_vocabulary), start=2)
#y_indices_to_len = dict(enumerate(y_vocabulary), start=2)

In [ ]:
y_indices_to_len

In [ ]:
def create_inverted_indices(X):
    return {v: k for k, v in X.items()}

In [ ]:
X_words_to_indices = create_inverted_indices(X_indices_to_words)
y_len_to_indices = create_inverted_indices(y_indices_to_len)

In [ ]:
y_len_to_indices

In [ ]:
print('Word index:', list(X_words_to_indices.items())[:3])
print('LEN index:', list(y_len_to_indices.items())[:3])

#### Encode lists - Convert to indices

In [ ]:
def encode_to_indices(X, X_words_to_indices, num_words=None):
    X_encoded = []
    for x in X:
        X_encoded_words = []
        if num_words:
            # We map the unknown words to the second first index of the matrix, for the test set
            X_encoded_words = list(map(lambda x: X_words_to_indices.get(x,1), x))
        else:
             X_encoded_words = list(map(X_words_to_indices.get, x))
            #for val in x:
            #    X_encoded_words.append(X_words_to_indices.get(val))
        X_encoded += [X_encoded_words]
    return X_encoded

In [ ]:
X_only_indices = encode_to_indices(X, X_words_to_indices)
y_only_indices = encode_to_indices(y, y_len_to_indices)
print('First sentences, word indices', X_only_indices[4])
print("")
print('First sentences, LEN indices', y_only_indices[4])

# Embeddings

In [ ]:
def most_similar_embeddings(embeddings_dict, key_word, n):
    
    d = {}
    key_word_embedding = embeddings_dict[key_word]
    
    for word, embedding in embeddings_dict.items():
        d_val = cosine_similarity([key_word_embedding], [embedding])[0][0]
        d[word] = d_val
    
    d_sorted = sorted(d.items(), key=itemgetter(1), reverse=True)[1:n+1] # Do not return table
        
    return d_sorted

In [ ]:
#table = most_similar_embeddings(embeddings_dict, 'table', 5)
#france = most_similar_embeddings(embeddings_dict, 'france', 5)
#sweden = most_similar_embeddings(embeddings_dict, 'sweden', 5)

In [ ]:
#print(table)
#print(france)
#print(sweden)

In [ ]:
def fill_glove_matrix(X_vocabulary, embeddings_dict):
    for word in X_vocabulary:
        if word in embeddings_dict:
            i = X_words_to_indices[word]
            embedding = embeddings_dict[word]
            word_embedding_matrix[i] = embedding
    return word_embedding_matrix

In [ ]:
word_embedding_matrix = np.random.random((len(X_vocabulary)+2, 100))
word_embedding_matrix = fill_glove_matrix(X_vocabulary, embeddings_dict)
print('Shape of embedding matrix:', word_embedding_matrix.shape)
#print('Embedding of the padding symbol, idx 0, random numbers', word_embedding_matrix[0])

# Padding the sequences

In [ ]:
max_seq_len = max(len(s) for s in X)
print("Maximum sentence length: ", max_seq_len)

In [ ]:
X_train = pad_sequences(X_only_indices, maxlen=max_seq_len)
y_train = pad_sequences(y_only_indices, maxlen=max_seq_len)

print(X_train[1])
print(y_train[1])

# The number of classes and 0 (padding symbol)
y_train = to_categorical(y_train, num_classes=nbr_of_classes + 1)
print(y_train[1])

# Load validation data

In [ ]:
X_val, y_val = extract_features(val_dict)

# We create the parallel sequences of indexes
X_only_indices_val = encode_to_indices(X_val, X_words_to_indices, num_words=total_word_count)
y_only_indices_val = encode_to_indices(y_val, y_len_to_indices)


X_val = pad_sequences(X_only_indices_val, maxlen=max_seq_len)
y_val = pad_sequences(y_only_indices_val, maxlen=max_seq_len)

print(X_val[1])
print(y_val[1])

y_val = to_categorical(y_val, num_classes=nbr_of_classes + 1)
print(y_val[1])

# Params

In [ ]:
epochs = 1
batch_size = 128

# Model

#### Simple RNN

In [ ]:
def build_simpleRNN():
    model = Sequential()
    model.add(Embedding(len(X_vocabulary)+2,
                               100,
                               mask_zero=True,
                               input_length=max_seq_len))
    model.layers[0].set_weights([word_embedding_matrix])
    # The default is True
    model.layers[0].trainable = True
    #model.add(layers.Bidirectional(layers.SimpleRNN(NB_CLASSES + 1, return_sequences=True)))
    #model.add(layers.Bidirectional(layers.LSTM(NB_CLASSES + 1, return_sequences=True)))
    model.add(SimpleRNN(32, return_sequences=True))
    #model.add(layers.Bidirectional(layers.LSTM(LSTM_UNITS, return_sequences=True)))
    model.add(Dense(nbr_of_classes + 1, activation='softmax'))
    return model

#### LSTM 

In [ ]:
def build_LSTM():
    model = Sequential()
    model.add(Embedding(len(X_vocabulary)+2,
                               100,
                               mask_zero=True,
                               input_length=max_seq_len))
    model.layers[0].set_weights([word_embedding_matrix])
    # The default is True
    model.layers[0].trainable = True
    #model.add(layers.Bidirectional(layers.SimpleRNN(NB_CLASSES + 1, return_sequences=True)))
    #model.add(layers.Bidirectional(layers.LSTM(NB_CLASSES + 1, return_sequences=True)))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dense(nbr_of_classes + 1, activation='softmax'))
    return model

#### LSTM BIDIRECTIONAL

#### GRU 

In [ ]:
model = build_simpleRNN()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(X_val, y_val))

# Evaluate

#### Load test set

In [ ]:
X_test, y_test = extract_features(test_dict)

# We create the parallel sequences of indexes
X_only_indices_test = encode_to_indices(X_test, X_words_to_indices, num_words=total_word_count)
y_only_indices_test = encode_to_indices(y_test, y_len_to_indices)

X_test_padded = pad_sequences(X_only_indices_test, maxlen=max_seq_len)
y_test_padded = pad_sequences(y_only_indices_test, maxlen=max_seq_len)

print(X_test_padded[1])
print(y_test_padded[1])

# The number of LEN classes and 0 (padding symbol)
y_test_vectorized = to_categorical(y_test_padded, num_classes=nbr_of_classes + 1)
print(y_test_vectorized[1])

In [ ]:
y_indices_to_len

In [ ]:
y_test[1]

In [ ]:
print(len(X_test), len(y_test), len(X_only_indices_test), len(y_only_indices_test))

In [ ]:
print(len(X_test_padded))

In [ ]:
loss, acc = model.evaluate(X_test_padded, y_test_vectorized, batch_size=batch_size, verbose=1)

In [ ]:
print(loss)

In [ ]:
print(acc)

#### Connleval

In [ ]:
corpus_pos_predictions = model.predict(X_test_padded)

In [ ]:
y_test_padded[77]

In [ ]:
corpus_pos_predictions[77]

In [ ]:
y_test[1]

In [ ]:
print('X_test', X_test[4])
print('X_test_padded', X_test_padded[4])
print('Y_test', y_test[4])
print('Y_test_padded', y_test_padded[4])
print('predictions', corpus_pos_predictions[4])

In [ ]:
pos_pred_num = []
for sent_nbr, sent_pos_predictions in enumerate(corpus_pos_predictions):
    #pos_pred_num += [sent_pos_predictions[-len(X_test[sent_nbr]):]]
    pos_pred_num += [sent_pos_predictions[-len(X_test[sent_nbr]):]]
#print(pos_pred_num[1])

In [ ]:
y_test[66]

In [ ]:
pos_pred = []
for sentence in pos_pred_num:
    pos_idx = list(map(np.argmax, sentence))
    pos_cat = list(map(y_indices_to_len.get, pos_idx))
    pos_pred += [pos_cat]

print(pos_pred[:4])
print("")
print(y_test[:4])

In [ ]:
len(pos_pred), len(y_test)

In [ ]:
c = 0
for arr in y_test:
    for x in arr:
        c += 1

print(c)

In [ ]:
pos_pred[1][2]

In [ ]:
c = 0
for i, arr in enumerate(y_test):
    for j, word in enumerate(arr):
        
        try:
            if pos_pred[i][j] == y_test[i][j]:
                c += 1
        except:
            print(i, j)

    
print(c)

In [ ]:
len(y_test[213]), len(y_test[213])

In [ ]:
total, correct, total_ukn, correct_ukn = 0, 0, 0, 0
for id_s, sentence in enumerate(X_test):
    for id_w, word in enumerate(sentence):
        
        if word not in X_words_to_indices:
            total += 1
            
print(total)

In [ ]:
total, correct, total_ukn, correct_ukn = 0, 0, 0, 0
missing = 0
for id_s, sentence in enumerate(X_test):
    for id_w, word in enumerate(sentence):
        
        try:
            if pos_pred[id_s][id_w] == y_test[id_s][id_w]:
                correct += 1
        except:
            print(id_s, id_w)
            missing += 1
        # The word is not in the dictionary
        if word not in X_words_to_indices:
            total_ukn += 1
            if pos_pred[id_s][id_w] == y_test[id_s][id_w]:
                correct_ukn += 1

total += correct
total += total_ukn
total += correct_ukn
print('total %d, correct %d, accuracy %f' % (total, correct, correct / total))
if total_ukn != 0:
    print('total unknown %d, correct %d, accuracy %f' % (total_ukn, correct_ukn, correct_ukn / total_ukn))

In [ ]:
f = open("F1_SCORE.txt", "w")

x = 0

for id_s, sentence in enumerate(y_test):
    
    if x == 1000:
        break
        
    for id_w, word in enumerate(sentence):
        x += 1
        
        try:
            
            word = test_dict[id_s][id_w]['form']
            ppos = test_dict[id_s][id_w]['ppos']
            pchunck = test_dict[i_s][id_w]['pchunk']
            y_true = y_test[id_s][id_w]
            y_pred = pos_pred[id_s][id_w]
  
            f.write(str(word) + " " + str(ppos) + " " + str(pchunck) + " " + str(y_true).upper() + " " + str(y_pred).upper() + "\n")
            
            if word == "-DOCSTART-" or word == ".":
                f.write("\n")
                
        except Exception as e:
            print(e)
            print(i_s, i_w)
        
f.close()
        

In [ ]:
pos_pred

In [ ]:
y_test